# Outline

* Iterable & Iterator
* Generator
* Simple Coroutines
* Native Coroutines

In [ ]:
import collections
from inspect import getgeneratorstate

# 1. Iterable & Iterators

## 1.1 Iterable

*Iterable* is anything you can loop over with a for loop.

* If an object implements the `__iter__()` magic method, it means it can be used in a for loop.


In [ ]:
# Basic objects in Python
a_list = [1, 2, 3]
a_tuple = (1, 2, 3)
a_set = {1, 2, 3}
a_str = "abcd"
a_dict = {"a":1, "b":2, "c":3}

obj_list = [a_list, a_tuple, a_set, a_str, a_dict]

In [ ]:
# Check if they have __iter__ method
for obj in obj_list:
    print(f"{type(obj)} has __iter__ method: {'__iter__' in dir(obj)}")

In [ ]:
# Check if they are iterable
for obj in obj_list:
    print(f"{type(obj)} is iterable: {isinstance(obj, collections.Iterable)}")

However, not all objects that work with for loop is an Iterable.

In [ ]:
# Create a class that only has the __getitem__ method
class FakeIterable:
    def __getitem__(self,index):
        if index < 5:
            return index
        else:
            raise IndexError

In [ ]:
# It can be iterated with for loop, but not an Iterable
a = FakeIterable()
for i in a:
    print(i)
print(isinstance(a, collections.Iterable))

## 1.2 Iterator

An *Iterator* is an object that only knows how to produce a series of values, one at a time, when it's being called by the already explored built-in `next()` function.

An Iterator is not necessarily an Iterable. See the example below:


In [ ]:
# Manually create an Iterator
class SequenceIterator:
    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step

    def __next__(self):
        value = self.current
        self.current += self.step
        return value

si = SequenceIterator(1, 2)

In [ ]:
# Use the next() to get one item at a time
next(si)

In [ ]:
# It is not Iterable because it does not implement __iter__()
for _ in si:
    pass

Iterables can be converted to Iterators by calling function `iter()`.

In [ ]:
# Check if they are iterator
for obj in obj_list:
    print(f"{type(obj)} is iterator: {isinstance(obj, collections.Iterator)}")

In [ ]:
# Convert iterable to iterator
a_iterator = iter(a_list)
print(f"a is a {type(a)}")
print(f"a_iterator is a {type(a_iterator)}")

In [ ]:
# Go through the sequence until exhausted
next(a_iterator)

In [ ]:
# Handle the StopIteration exception
next(a_iterator, "The End")

## 1.3 Summary

Iterable implements `__iter__`, while Iterator implements `__next__`.

* An Iterable is not necessarily Iterator.
* An Iterator is not necessarily Iterable.
* Iterable can be conveniently converted to Iterator by calling `iter()`.

# 2. Generators

A *Generator* is both an Iterable and an Iterator.

Generators were introduced in Python with the idea of introducing iteration while improving the performance of the program.

* save memory by producing each partidular element one at a time
* lazy computations allow for infinit sequences

## 2.1 Create Generators


In [ ]:
# Create a generator with yield
def mygen():
    x = 0
    while x < 10:
        yield x**2
        x += 1

gen_1 = mygen()

# Create a generator with generator expression
gen_2 = (x**2 for x in range(10))

In [ ]:
print(type(gen_1))
print(type(gen_2))

## 2.2 How do generators work

Generators pause after yield statement, and can be re-activated by `next()`.

In [ ]:
def mygen():
    x = 0
    while x < 10:
        print(f"State before yield: {getgeneratorstate(gen)}")
        yield x**2
        print(f"State after yield: {getgeneratorstate(gen)}")
        x += 1

In [ ]:
# Create a generator
gen = mygen()
print(getgeneratorstate(gen))

In [ ]:
next(gen)

In [ ]:
print(getgeneratorstate(gen))

## 2.3 Memory saving with generators

In [ ]:
%%writefile memory.py
from memory_profiler import profile
import random

def int_list(n):
    result = []
    for i in range(n):
        result.append(random.randint(0, n))
    return result

def int_generator(n):
    for i in range(n):
        yield random.randint(0, n)
 
@profile
def main():
    n = 1000000
    int_1 = int_generator(n)
    int_2 = int_list(n)
    int_3 = list(int_generator(n))
 
if __name__ == '__main__':
    main()

## 2.4 Summary

Borrow a nice picture that perfectly summarizes the relation among core concepts (forget about the example that an Iterator is not necessarily an Iterable).

![alt text](https://th.bing.com/th/id/R95b4076d30e55da078045cdade28cea3?rik=6CiNzZWhpII7RA&riu=http%3a%2f%2fnvie.com%2fimg%2frelationships.png&ehk=%2be4cF8sjtgZYVwbqYK%2fRkRFwauNqCqM%2fy%2bmlaQxAw3Y%3d&risl=&pid=ImgRaw)

# 3. Coroutines via Enhanced Generators

Can simple generators ...

* suspend? (YES)
* send/receive data from the context? (NO)
* handle exceptions from the caller's context? (NO)

New methods to enhance the generators:

* .close()
* .throw(exception)
* .send(value)

Generators and coroutines are syntactically (and technically) the same, but semantically, they are different. 

* We create generators when we want to achieve efficient iteration. 
* We typically create coroutines with the goal of running non-blocking I/O operations.

## 3.1 Enhanced generators



In [ ]:
# A sequence with user-defined step
def jumping_step():
    index = 0
    while True:
        jump = yield index
        if jump is None:
            jump = 1
        index += jump

In [ ]:
itr = jumping_step()
type(itr)

In [ ]:
next(itr)

In [ ]:
itr.send(56)

## 3.2 Coroutines

In [ ]:
def char_freq(frec_dict):

    while True:
        word = yield
        word = word.lower()
        for char in word:
            if frec_dict.get(char) == None:
                frec_dict[char] = 1
            else:
                frec_dict[char] += 1

frec_dict = {}
counter = char_freq(frec_dict)

In [ ]:
next(counter)
frec_dict

In [ ]:
counter.send("iterable")
frec_dict

# 4. Native Coroutines

Coroutines with async and await syntax

In [ ]:
import time,asyncio

async def count():
    print("count 1")
    await asyncio.sleep(1)
    print("count 4")

async def count_further():
    print("count 2")
    await asyncio.sleep(1)
    print("count 5")

async def count_even_further():
    print("count 3")
    await asyncio.sleep(1)
    print("count 6")

async def main():
    await asyncio.gather(count(), count_further(), count_even_further())

s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")

# 5. Links

* [Demystifying Coroutines and Asynchronous Programming in Python](https://www.youtube.com/watch?v=7AoANOGIDuM)

* [Python 101: iterators, generators, coroutines](https://www.integralist.co.uk/posts/python-generators/)

* [Python Generators/Coroutines/Async IO with examples](https://medium.com/analytics-vidhya/python-generators-coroutines-async-io-with-examples-28771b586578)

